In [1]:
import warnings
warnings.filterwarnings("ignore")
%load_ext autoreload
%autoreload 2
from qiskit_metal import Dict, MetalGUI, designs

from qiskit_metal.qlibrary.qubits.transmon_pocket import TransmonPocket
from qiskit_metal.qlibrary.qubits.transmon_pocket_6 import TransmonPocket6
from qiskit_metal.qlibrary.tlines.meandered import RouteMeander
from qiskit_metal.qlibrary.tlines.straight_path import RouteStraight
from qiskit_metal.qlibrary.terminations.launchpad_wb_coupled import LaunchpadWirebondCoupled
from qiskit_metal.qlibrary.terminations.short_to_ground import ShortToGround
from qiskit_metal.qlibrary.couplers.coupled_line_tee import CoupledLineTee
from qiskit_metal.qlibrary.user_components.left_coupler import LeftCoupler
from qiskit_metal.qlibrary.user_components.right_coupler import RightCoupler
from qiskit_metal.qlibrary.user_components.straightdown_coupler import StraightDownCoupler
from qiskit_metal.qlibrary.user_components.straightup_coupler import StraightUpCoupler

In [2]:
design = designs.DesignFlipChip({}, overwrite_enabled=True)
gui = MetalGUI(design)

In [3]:
design.chips['C_chip'].size['size_x']='3mm'
design.chips['C_chip'].size['size_y']='3mm'
design.chips['Q_chip'].size['size_x']='2mm'
design.chips['Q_chip'].size['size_y']='2mm'
design.chips['Q_chip'].size['center_z']='7um'

In [4]:
design.get_chip_z('Q_chip')

'7um'

In [5]:

cp0 = dict(down = dict(loc_W=1, loc_H=-1, pad_width='120um'))
q0=[TransmonPocket6(design, 'Q0', options=dict(pos_x='-300um',
                                            pos_y='-500um',
                                            chip='Q_chip'))]
                                            
qubits=q0
q0x=float(qubits[0].options.pos_x.strip('um'))
q0y=float(qubits[0].options.pos_y.strip('um'))


c0 = StraightUpCoupler(design, "c0", options=dict(pos_x=f"{q0x-150}um",
                                              pos_y=f"{q0y+140}um",
                                              pad_width="150um",
                                              pad_height="50um",
                                              cpw_stub_height="200um",
                                              chip="C_chip"))
design.rebuild()
gui.rebuild()
gui.autoscale()

In [6]:
lpL=LaunchpadWirebondCoupled(design, 'lpL',
                                         options = dict(chip="C_chip",
                                         pos_x = '-700um',
                                         pos_y = '800um',
                                         lead_length = '30um'))
lpR=LaunchpadWirebondCoupled(design, 'lpR',
                                         options = dict(chip="C_chip",
                                         pos_x = '700um',
                                         pos_y = '800um',
                                         orientation='180',
                                         lead_length = '30um'))                                        

In [7]:
stg=ShortToGround(design, 'stg', options=Dict(chip="C_chip",
                                                pos_x='0.65mm',
                                                pos_y='-0.2mm',
                                                orientation='270'))

In [8]:
fl0=RouteStraight(design, 'fl0',options=Dict(chip="C_chip", pin_inputs=Dict(start_pin=Dict(
                                                            component='lpL',
                                                            pin='tie'),
                                                            end_pin=Dict(
                                                            component='lpR',
                                                            pin='tie'))))


# Readout resonator

In [9]:
from collections import OrderedDict
jogs=OrderedDict()
def connect(component_name: str, component1: str, pin1: str, component2: str, pin2: str,
            length: str, start_jogged_extension: OrderedDict, end_jogged_extension: OrderedDict,
            spacing: str, asymmetry='0 um',  start_straight='0 um', end_straight='0 um', flip=False):
    """Connect two pins with a CPW."""
    myoptions = Dict(
        chip="C_chip",
        pin_inputs=Dict(
            start_pin=Dict(
                component=component1,
                pin=pin1),
            end_pin=Dict(
                component=component2,
                pin=pin2)),
        lead=Dict(
            start_straight=start_straight,
            end_straight=end_straight,
            start_jogged_extension=start_jogged_extension,
            end_jogged_extension=end_jogged_extension),
        total_length=length,
        fillet = '99.9um',
        meander=dict(asymmetry=asymmetry,
                     spacing=spacing))
    myoptions.meander.lead_direction_inverted = 'true' if flip else 'false'
    return RouteMeander(design, component_name, myoptions)

# Resonator length

In [10]:
from qiskit_metal.analyses.em.cpw_calculations import guided_wavelength

def find_resonator_length(frequency, line_width, line_gap, N): 
    [lambdaG, etfSqrt, q] = guided_wavelength(frequency*10**9, line_width*10**-6,
                                              line_gap*10**-6, 750*10**-6, 200*10**-9)
    return str(lambdaG/N*10**3)+" mm"

In [11]:
resonator_length_4=[]
a=find_resonator_length(7.5,10,6,4)
a=a.strip('mm')
a=float(a)-0.25 # subtract coupler length
resonator_length_4.append(a)
print(resonator_length_4)


[3.807562247348839]


In [ ]:
jogs1=OrderedDict()
jogs1[0]=['R','300um']
jogs1[1]=['R', '300um']
jogs1[2]=['L','800um']
jogs1[3]=['R','200um']
jogs1[4]=['R','600um']
jogs1[5]=['L','200um']
jogs1[6]=['L','600um']


In [ ]:
rr0=connect('rr0', 'c0', 'in','stg', 'short',
            resonator_length_4[0],jogs1,jogs, 
            '100um','0um', '918um', '100um')
            

# EPR Analysis

In [14]:
from qiskit_metal.analyses.quantization import EPRanalysis
eig_rd = EPRanalysis(design, "hfss")
hfss = eig_rd.sim.renderer

In [15]:
hfss.start()

INFO 11:50AM [connect_project]: Connecting to Ansys Desktop API...
INFO 11:50AM [load_ansys_project]: 	Opened Ansys App
INFO 11:50AM [load_ansys_project]: 	Opened Ansys Desktop v2018.0.0
INFO 11:50AM [load_ansys_project]: 	Opened Ansys Project
	Folder:    C:/Users/kimje/Documents/Ansoft/
	Project:   KPS_Planar
INFO 11:50AM [connect_design]: 	Opened active design
	Design:    Planar [Solution type: Eigenmode]
INFO 11:50AM [get_setup]: 	Opened setup `Setup`  (<class 'pyEPR.ansys.HfssEMSetup'>)
INFO 11:50AM [connect]: 	Connected to project "KPS_Planar" and design "Planar" 😀 



True

In [16]:
hfss.activate_ansys_design("flip7um_PlanarDesign", 'eigenmode')
hfss.render_design(['Q0','c0','lpL','lpR','stg','rr0','fl0'],[],
                    box_plus_buffer=False)

11:51AM 00s WARNING [activate_ansys_design]: The design_name=flip7um_PlanarDesign was not in active project.  Designs in active project are: 
['Planar', 'Design_q3d'].  A new design will be added to the project.  
INFO 11:51AM [connect_design]: 	Opened active design
	Design:    flip7um_PlanarDesign [Solution type: Eigenmode]
WARNING 11:51AM [connect_setup]: 	No design setup detected.
WARNING 11:51AM [connect_setup]: 	Creating eigenmode default setup.
INFO 11:51AM [get_setup]: 	Opened setup `Setup`  (<class 'pyEPR.ansys.HfssEMSetup'>)


In [17]:
hfss.close()


True

# LOM Analysis

In [18]:
from qiskit_metal.analyses.quantization import LOManalysis
c1 = LOManalysis(design, "q3d")

In [19]:
c1.sim.setup.max_passes = 10
c1.sim.setup_update(freq_ghz='4.78',solution_order = 'Medium', auto_increase_solution_order = 'False')
c1.sim.setup

{'name': 'Setup',
 'reuse_selected_design': True,
 'reuse_setup': True,
 'freq_ghz': '4.78',
 'save_fields': False,
 'enabled': True,
 'max_passes': 10,
 'min_passes': 2,
 'min_converged_passes': 2,
 'percent_error': 0.5,
 'percent_refinement': 30,
 'auto_increase_solution_order': 'False',
 'solution_order': 'Medium',
 'solver_type': 'Iterative'}

In [21]:
#c1.sim.run(components=['Q0','c0'], open_terminations=[('c0', 'in')])
c1.sim._get_results_from_renderer()
c1.sim.capacitance_matrix

INFO 03:02PM [get_matrix]: Exporting matrix data to (C:\Users\kimje\AppData\Local\Temp\tmp6_uopujx.txt, C, , Setup:LastAdaptive, "Original", "ohm", "nH", "fF", "mSie", 4780000000, Maxwell, 1, False
INFO 03:02PM [get_matrix]: Exporting matrix data to (C:\Users\kimje\AppData\Local\Temp\tmp_9jc93bs.txt, C, , Setup:AdaptivePass, "Original", "ohm", "nH", "fF", "mSie", 4780000000, Maxwell, 1, False
INFO 03:02PM [get_matrix]: Exporting matrix data to (C:\Users\kimje\AppData\Local\Temp\tmpf014enof.txt, C, , Setup:AdaptivePass, "Original", "ohm", "nH", "fF", "mSie", 4780000000, Maxwell, 2, False
INFO 03:02PM [get_matrix]: Exporting matrix data to (C:\Users\kimje\AppData\Local\Temp\tmpjbpup675.txt, C, , Setup:AdaptivePass, "Original", "ohm", "nH", "fF", "mSie", 4780000000, Maxwell, 3, False
INFO 03:02PM [get_matrix]: Exporting matrix data to (C:\Users\kimje\AppData\Local\Temp\tmp6sdx2hm3.txt, C, , Setup:AdaptivePass, "Original", "ohm", "nH", "fF", "mSie", 4780000000, Maxwell, 4, False
INFO 03:02

,ground,pad_bot_Q0,pad_top_Q0,quad_coupler_c0
ground,375.810,-120.45000,-117.1900,-74.51400
pad_bot_Q0,-120.450,150.45000,-28.0200,-0.36257
pad_top_Q0,-117.190,-28.02000,150.4100,-3.61640
quad_coupler_c0,-74.514,-0.36257,-3.6164,78.50400


In [22]:
c1.setup.junctions = Dict({'Lj': 11, 'Cj': 2})
c1.setup.freq_readout = 7.54
c1.setup.freq_bus = []
c1.run_lom()
c1.lumped_oscillator_all

[1, 2] [3]
Predicted Values

Transmon Properties
f_Q 4.821870 [GHz]
EC 214.213804 [MHz]
EJ 14.854145 [GHz]
alpha -239.885117 [MHz]
dispersion 0.373370 [KHz]
Lq 10.995564 [nH]
Cq 90.424742 [fF]
T1 5851.503645 [us]

**Coupling Properties**

tCqbus1 -1.626966 [fF]
gbus1_in_MHz -20.926645 [MHz]
χ_bus1 -0.027533 [MHz]
1/T1bus1 27.198982 [Hz]
T1bus1 5851.503645 [us]
Bus-Bus Couplings


,fQ,EC,EJ,alpha,dispersion,gbus,chi_in_MHz,χr MHz,gr MHz
1,5.852773,322.861486,14.854145,-373.633026,25.65302,[-56.86058450409791],[-0.7086822159802318],0.708682,56.860585
2,5.163388,247.45048,14.854145,-279.997838,1.833232,[-21.212867638821002],[-0.041508141324023126],0.041508,21.212868
3,4.938422,225.257045,14.854145,-253.134647,0.659001,[-19.11715051460557],[-0.026126007363223198],0.026126,19.117151
4,4.859836,217.777274,14.854145,-244.151983,0.450645,[-19.690526522766895],[-0.025411741983867458],0.025412,19.690527
5,4.83796,215.719977,14.854145,-241.687601,0.404501,[-20.522085185047676],[-0.026949145486487393],0.026949,20.522085
6,4.82187,214.213804,14.854145,-239.885117,0.37337,[-20.926645318624267],[-0.02753343504253757],0.027533,20.926645


In [23]:
c1.sim.close()

Warning! 3 COM references still alive
Ansys will likely refuse to shut down
